### Rewriting code from RDA_type_seasonal.ipynb using some of the tricks I've learned

In [1]:
import netCDF4 as nc
import numpy as np
import time
import os

In [2]:
ferretpath = "/Users/Siwen/Desktop/ferret/bin"
rdapath = "APHRO_ZH_front_025deg_V1101."

In [3]:
## year = 2007
## ff = filename = ferretpath + '/' + rdapath + str(year) + '.nc'
## tst = nc.Dataset(ff, 'r')
## tst.variables

In [4]:
## start by loading all precipitation data for each year.
P = np.zeros((365,80,72,57))
P_band = np.zeros((365,80,72,57))
P_local = np.zeros((365,80,72,57))

for yr in range(1951,2008):
    
    #Build filename
    filename = ferretpath + '/' + rdapath + str(yr) + '.nc'
    print(filename),
    myf = nc.Dataset(filename, 'r')
    
    #Obtain fraction of different types of rainfall
    P[:,:,:,yr-1951] = myf['P'][:365,:,:]
    P_band[:,:,:,yr-1951] = myf['P_band1'][:365,:,:] + myf['P_band2'][:365,:,:]
    P_local[:,:,:,yr-1951] = myf['P_leftover'][:365,:,:]
    
    myf.close()

/Users/Siwen/Desktop/ferret/bin/APHRO_ZH_front_025deg_V1101.1951.nc
/Users/Siwen/Desktop/ferret/bin/APHRO_ZH_front_025deg_V1101.1952.nc
/Users/Siwen/Desktop/ferret/bin/APHRO_ZH_front_025deg_V1101.1953.nc
/Users/Siwen/Desktop/ferret/bin/APHRO_ZH_front_025deg_V1101.1954.nc
/Users/Siwen/Desktop/ferret/bin/APHRO_ZH_front_025deg_V1101.1955.nc
/Users/Siwen/Desktop/ferret/bin/APHRO_ZH_front_025deg_V1101.1956.nc
/Users/Siwen/Desktop/ferret/bin/APHRO_ZH_front_025deg_V1101.1957.nc
/Users/Siwen/Desktop/ferret/bin/APHRO_ZH_front_025deg_V1101.1958.nc
/Users/Siwen/Desktop/ferret/bin/APHRO_ZH_front_025deg_V1101.1959.nc
/Users/Siwen/Desktop/ferret/bin/APHRO_ZH_front_025deg_V1101.1960.nc
/Users/Siwen/Desktop/ferret/bin/APHRO_ZH_front_025deg_V1101.1961.nc
/Users/Siwen/Desktop/ferret/bin/APHRO_ZH_front_025deg_V1101.1962.nc
/Users/Siwen/Desktop/ferret/bin/APHRO_ZH_front_025deg_V1101.1963.nc
/Users/Siwen/Desktop/ferret/bin/APHRO_ZH_front_025deg_V1101.1964.nc
/Users/Siwen/Desktop/ferret/bin/APHRO_ZH_front_0

In [ ]:
## PLOTTING - target is an 8-panel figure for each of the different potential metrics.

## create one big figure behind subpanels
fig, axes = plt.subplots(4,2,figsize=(12,12), facecolor='w')
st = plt.suptitle("Alternative Rainfall Metrics $M_1-M_8$, 1980-2007 versus 1951-1979", size = 20)
#st.set_font_properties(font1)

## lists of the characteristics for each subplot - what column is being plotted, title and labels.
mets = ['M_1','M_2','M_3','M_4','M_5','M_6','M_7','M_8']
titles = ["$M_1$ - Latitude of Max Rainfall","$M_2$ - Rainfall Centroid Lat","$M_3$ - Intensity of Max Rainfall",\
          "$M_4$ - Mean All-China Rainfall","$M_5$ - Mean North China Rainfall","$M_6$ - Mean South China Rainfall",\
          "$M_7$ - North China Rainfall Frequency","$M_8$ - South China Rainfall Frequency"]
xlabels = ['','','','','','','Day of Year','Day of Year']
ylabels = ["Latitude ($^{\circ}$)","Latitude ($^{\circ}$)","mm day$^{-1}$","mm day$^{-1}$","mm day$^{-1}$",\
           "mm day$^{-1}$","Frequency","Frequency"]

## cycle through each set of axes and make correct plot
for ax, title, xlabel, ylabel, met in zip(axes.reshape(-1),titles,xlabels,ylabels,mets):
    sns.tsplot(data = met_df, ax = ax, value = met, time = 'day', unit = 'year', condition = 'condition1', ci = 95, legend = False, color=colors1)
    ax.set_title(title, fontsize=16)
    ax.set_xlabel(xlabel, fontsize=13, labelpad=0.5)
    ax.set_ylabel(ylabel, fontsize=13, labelpad=0.5)
    
## Custom edits to specific plots that don't look right
axes[0,1].set_yticks(np.linspace(27,31,5))
axes[3,1].set_ylim([0,1])

## set legend
lines = axes[0,0].get_lines()
leg = fig.legend(lines, ('1951-1979', '1980-2007'), loc="center", ncol=1, frameon = True, prop={'size': 13},\
                 bbox_to_anchor=[0.52, 0.48])
leg.get_frame().set_edgecolor('k')
leg.get_frame().set_facecolor('w')

##reframe figure
plt.tight_layout()
st.set_y(0.99)
fig.subplots_adjust(top=0.94)